In [ ]:
import sqlite3
from sqlite3 import Error
import requests

In [ ]:
# https://catalog.data.gov/dataset/air-quality-measures-on-the-national-environmental-health-tracking-network

### Get dataset

In [ ]:
dataset_url_path = 'https://data.cdc.gov/api/views/cjae-szjv/rows.json?accessType=DOWNLOAD'

In [ ]:
resp = requests.get(dataset_url_path)

In [ ]:
resp.status_code

In [ ]:
data = resp.json()

In [ ]:
len(data)

In [ ]:
data.keys()

In [ ]:
column_names = [column['fieldName'].replace(':', '') for column in data['meta']['view']['columns']]

### Create connection to db

In [ ]:
def create_connection(path):
    """ 
    Create a database connection to the SQLite database specified by path.
    """
    connection = None
    try:
        connection = sqlite3.connect(path)
        #print("Connection to SQLite DB successful")
    except Error as e:
        print(e)
    return connection

In [ ]:
#connection = create_connection("E:\\sm_app.sqlite")

### Create table

In [ ]:
def create_table(connection, create_table_sql):
    """ 
    Create a table from the create_table_sql statement
    """
    try:
        c = connection.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

In [ ]:
def main():
    database = "pythonsqlite.db"

    sql_create_measures_table = """CREATE TABLE IF NOT EXISTS measures (
                                    sid text PRIMARY KEY,
                                    id text NOT NULL,
                                    position integer NOT NULL,
                                    created_at integer NOT NULL,
                                    created_meta NULL,
                                    updated_at integer NOT NULL,
                                    updated_meta NULL,
                                    meta text NOT NULL,
                                    measureid text NOT NULL,
                                    measurename text NOT NULL,
                                    measuretype text NOT NULL,
                                    stratificationlevel text NOT NULL,
                                    statefips text NOT NULL,
                                    statename text NOT NULL,
                                    countyfips text NOT NULL,
                                    countyname text NOT NULL,
                                    reportyear text NOT NULL,
                                    value text NOT NULL,
                                    unit text NOT NULL,
                                    unitname text NOT NULL,
                                    dataorigin text NOT NULL,
                                    monitoronly text NOT NULL
                                );"""
    
    #FOREIGN KEY (project_id) REFERENCES projects (id)
    
    # create a database connection
    connection = create_connection(database)

    # create table
    if connection is not None:
        # create measures table
        create_table(connection, sql_create_measures_table)
    else:
        print("Error! cannot create the database connection.")

In [ ]:
if __name__ == '__main__':
    main()

### Insert measure

In [ ]:
def create_measure(connection, project):
    """
    A function to insert a new measure into the measures table.    
    Use the  lastrowid attribute of the Cursor object to get back the generated id.
    """
    sql = ''' INSERT INTO measures(
                            sid,
                            id,
                            position,
                            created_at,
                            created_meta,
                            updated_at,
                            updated_meta,
                            meta,
                            measureid,
                            measurename,
                            measuretype,
                            stratificationlevel,
                            statefips,
                            statename,
                            countyfips,
                            countyname,
                            reportyear,
                            value,
                            unit,
                            unitname,
                            dataorigin,
                            monitoronly
                            )
              VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) '''
    cursor = connection.cursor()
    cursor.execute(sql, test_measure)
    connection.commit()
    return cursor.lastrowid

In [ ]:
test_measure = data['data'][1]

In [ ]:
def main():
    database = "pythonsqlite.db"

    # create a database connection
    connection = create_connection(database)
    with connection:
        # create a new measure
        project = ('Cool App with SQLite & Python', '2015-01-01', '2015-01-30');
        project_id = create_measure(connection, test_measure)

In [ ]:
if __name__ == '__main__':
    main()

### Querying measures

In [ ]:
def select_all_measures(connection):
    """
    Query all rows in the measures table
    :param conn: the Connection object
    :return:
    """
    cursor = connection.cursor()
    cursor.execute("SELECT * FROM measures")

    rows = cursor.fetchall()
    
    names = list(map(lambda x: x[0], cursor.description))
    print(names)

    for row in rows:
        print(row)

In [ ]:
def main():
    database = "pythonsqlite.db"

    # create a database connection
    connection = create_connection(database)
    with connection:
        select_all_measures(connection)

In [ ]:
if __name__ == '__main__':
    main()